# RBRP Dry Protocol - バイオインフォマティクス解析自動化パイプライン

このノートブックは、Eric Koolラボの論文「Reactivity-based RNA profiling for analyzing transcriptome interactions of small molecules in human cells」のドライプロトコル部分を自動化します。

**対象ユーザー**: バイオインフォマティクス初学者  
**入力**: FASTQファイル（ペアエンド対応）  
**出力**: RNA-薬物相互作用解析結果

## 🔧 モダンシーケンサー対応
最近のシーケンサー（Illumina NovaSeq、NextSeq等）では以下の処理が自動で実行されるため、スキップオプションを用意しています：

- **デマルチプレックス**: バーコード分離済みFASTQファイルの出力
- **アダプタートリミング**: アダプター配列除去済みの出力
- **品質フィルタリング**: 低品質リード除去済みの出力

デフォルト設定では、これらの処理をスキップして高速化されています。

## 使用方法
1. 設定セクションで入力ファイルパスを指定
2. スキップオプションを必要に応じて調整
3. 「Run All Cells」で全自動実行
4. 結果は`data/results/`フォルダに保存されます

## 1. 環境設定と初期化

In [1]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import logging
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートディレクトリを設定
PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT / 'src'))

# ログ設定
log_file = PROJECT_ROOT / 'logs' / f'rbrp_pipeline_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print(f"🧬 RBRP Dry Protocol Pipeline Started")
print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📝 Log File: {log_file}")

🧬 RBRP Dry Protocol Pipeline Started
📁 Project Root: /home/akahod3f/work/kool
📝 Log File: /home/akahod3f/work/kool/logs/rbrp_pipeline_20250917_044525.log


## 2. 設定ファイル読み込み・入力ファイル指定

In [ ]:
# 🔧 処理スキップオプション
PROCESSING_OPTIONS = {
    'skip_demultiplex': True,            # True: デマルチプレックスをスキップ
    'skip_adapter_trimming': True,       # True: アダプタートリミングをスキップ
    'skip_pcr_duplicate_removal': False, # True: PCR重複除去をスキップ
    'perform_quality_control': False,    # True: FastQCによる品質確認を実行
    'adapter_sequences': {               # カスタムアダプター配列（trimming実行時のみ使用）
        'R1': 'AGATCGGAAGAGCGGTTCAG',
        'R2': 'AGATCGGAAGAGCGGTTCAG'
    },
    # 💾 ディスク容量管理オプション
    'cleanup_intermediate_fastq': True,  # True: 中間FASTQファイルを自動削除してディスク容量節約
    'preserve_original_fastq': True,     # True: 元のダウンロード/シーケンスファイルは保持
    'cleanup_confirmation': True         # True: 削除前に安全性確認
}

## 3. 依存関係チェック・外部ツール確認

In [ ]:
def check_external_tools():
    """外部ツールの存在確認"""
    required_tools = [
        'fastqc',
        'bowtie2',
        'gffread',
        'wiggletools',
        'bedGraphToBigWig',
        'wigToBigWig'
    ]
    
    missing_tools = []
    
    for tool in required_tools:
        try:
            result = subprocess.run(['which', tool], capture_output=True, text=True)
            if result.returncode == 0:
                print(f"✅ {tool}: {result.stdout.strip()}")
            else:
                missing_tools.append(tool)
                print(f"❌ {tool}: 見つかりません")
        except Exception as e:
            missing_tools.append(tool)
            print(f"❌ {tool}: エラー - {e}")
    
    if missing_tools:
        print(f"\n⚠️ 以下のツールが不足しています: {', '.join(missing_tools)}")
        print("インストール方法はREADME.mdを参照してください")
        return False
    else:
        print("\n🎉 すべての必要ツールが利用可能です")
        return True

tools_available = check_external_tools()

## 4. パイプライン実行関数群

In [ ]:
def run_command(cmd, description, check=True):
    """コマンド実行のヘルパー関数"""
    logger.info(f"実行中: {description}")
    logger.debug(f"コマンド: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, check=check, 
                              capture_output=True, text=True)
        if result.returncode == 0:
            logger.info(f"✅ 完了: {description}")
            return result
        else:
            logger.error(f"❌ 失敗: {description}")
            logger.error(f"エラー出力: {result.stderr}")
            if check:
                raise subprocess.CalledProcessError(result.returncode, cmd)
            return result
    except Exception as e:
        logger.error(f"❌ 例外発生: {description} - {e}")
        if check:
            raise
        return None

def is_original_fastq_file(file_path, sample_id):
    """
    元のFASTQファイル（ダウンロード/シーケンスファイル）かどうか判定
    
    Args:
        file_path (Path): ファイルパス
        sample_id (str): サンプルID
    
    Returns:
        bool: 元ファイルの場合True
    """
    if sample_id not in INPUT_FASTQ_FILES:
        return False
        
    original_r1 = Path(INPUT_FASTQ_FILES[sample_id]['R1']).resolve()
    original_r2 = Path(INPUT_FASTQ_FILES[sample_id]['R2']).resolve()
    file_path_resolved = Path(file_path).resolve()
    
    return file_path_resolved == original_r1 or file_path_resolved == original_r2

def safe_cleanup_fastq_files(files_to_remove, sample_id, step_description):
    """
    中間FASTQファイルを安全に削除
    
    Args:
        files_to_remove (list): 削除対象ファイルリスト
        sample_id (str): サンプルID
        step_description (str): 処理ステップの説明
    
    Returns:
        dict: 削除結果の統計
    """
    if not PROCESSING_OPTIONS.get('cleanup_intermediate_fastq', False):
        return {'skipped': True, 'reason': 'cleanup disabled'}
    
    cleanup_stats = {
        'files_checked': 0,
        'files_removed': 0,
        'space_freed_mb': 0,
        'preserved_originals': 0,
        'errors': []
    }
    
    for file_path in files_to_remove:
        file_path = Path(file_path)
        cleanup_stats['files_checked'] += 1
        
        if not file_path.exists():
            continue
            
        # 元ファイル保護チェック
        if PROCESSING_OPTIONS.get('preserve_original_fastq', True):
            if is_original_fastq_file(file_path, sample_id):
                cleanup_stats['preserved_originals'] += 1
                logger.info(f"🔒 元ファイル保護: {file_path.name}")
                continue
        
        # シンボリックリンクの場合は削除（実ファイルは削除しない）
        if file_path.is_symlink():
            try:
                file_size_mb = 0  # シンボリックリンクなのでサイズはカウントしない
                file_path.unlink()
                cleanup_stats['files_removed'] += 1
                logger.info(f"🗑️ シンボリックリンク削除: {file_path.name}")
            except Exception as e:
                cleanup_stats['errors'].append(f"シンボリックリンク削除失敗 {file_path}: {e}")
            continue
        
        # 安全性確認
        if PROCESSING_OPTIONS.get('cleanup_confirmation', True):
            # ファイルが中間処理ファイルかどうか確認
            is_intermediate = any(keyword in str(file_path) for keyword in 
                                ['_demux', '_rmdup', '_trimmed', 'processed/'])
            
            if not is_intermediate:
                logger.warning(f"⚠️ 削除スキップ（中間ファイルでない可能性）: {file_path}")
                continue
        
        # ファイルサイズ記録
        try:
            file_size_mb = file_path.stat().st_size / (1024 * 1024)
            
            # ファイル削除実行
            file_path.unlink()
            cleanup_stats['files_removed'] += 1
            cleanup_stats['space_freed_mb'] += file_size_mb
            
            logger.info(f"🗑️ 削除完了: {file_path.name} ({file_size_mb:.1f} MB)")
            
        except Exception as e:
            cleanup_stats['errors'].append(f"削除失敗 {file_path}: {e}")
            logger.error(f"❌ ファイル削除エラー: {file_path} - {e}")
    
    # クリーンアップサマリー
    if cleanup_stats['files_removed'] > 0:
        logger.info(f"💾 {step_description} - クリーンアップ完了: "
                   f"{cleanup_stats['files_removed']}ファイル削除, "
                   f"{cleanup_stats['space_freed_mb']:.1f}MB節約")
    
    return cleanup_stats

def get_condition_replicates(condition_name):
    """
    実験条件のreplicate一覧を取得
    
    Args:
        condition_name (str): 実験条件名
    
    Returns:
        list: replicate ID一覧
    """
    if condition_name in EXPERIMENT_DESIGN['conditions']:
        return EXPERIMENT_DESIGN['conditions'][condition_name]['replicates']
    return []

def get_samples_by_type(sample_type):
    """
    サンプルタイプ別にreplicate一覧を取得
    
    Args:
        sample_type (str): 'probe' または 'control'
    
    Returns:
        list: replicate ID一覧
    """
    samples = []
    for condition_name, condition_info in EXPERIMENT_DESIGN['conditions'].items():
        if condition_info['type'] == sample_type:
            samples.extend(condition_info['replicates'])
    return samples

def get_condition_from_sample(sample_id):
    """
    サンプルIDから実験条件を取得
    
    Args:
        sample_id (str): サンプルID
    
    Returns:
        str: 実験条件名
    """
    if sample_id in INPUT_FASTQ_FILES:
        return INPUT_FASTQ_FILES[sample_id]['condition']
    return None

def merge_replicate_files(condition_name, file_pattern, output_file, method='mean'):
    """
    同一条件のreplicateファイルを統計的にマージ
    
    Args:
        condition_name (str): 実験条件名
        file_pattern (str): ファイルパターン ('rt', 'rpkm', 'rbrp' など)
        output_file (Path): 出力ファイルパス
        method (str): 統計手法 ('mean', 'median', 'sum')
    
    Returns:
        bool: 成功/失敗
    """
    replicates = get_condition_replicates(condition_name)
    
    if len(replicates) < EXPERIMENT_DESIGN['replicate_handling']['min_replicates']:
        logger.warning(f"⚠️ {condition_name}: replicate数不足 ({len(replicates)} < {EXPERIMENT_DESIGN['replicate_handling']['min_replicates']})")
        return False
    
    # 各replicateのファイルパス取得
    input_files = []
    for replicate_id in replicates:
        if file_pattern == 'rt':
            file_path = find_latest_processed_files(replicate_id, "rt")
        elif file_pattern == 'rpkm':
            file_path = PROJECT_ROOT / f"data/processed/aligned/{replicate_id}.rpkm"
        elif file_pattern == 'rbrp':
            file_path = PROJECT_ROOT / f"data/processed/rbrp_scores/{replicate_id}_filtered.out"
        else:
            continue
            
        if file_path and Path(file_path).exists():
            input_files.append(str(file_path))
        else:
            logger.warning(f"⚠️ {replicate_id}: {file_pattern}ファイルが見つかりません")
    
    if len(input_files) < 2:
        logger.warning(f"⚠️ {condition_name}: 有効な{file_pattern}ファイルが不足")
        return False
    
    # Pythonスクリプトでreplicate統合
    merge_cmd = f"""python {PROJECT_ROOT}/scripts/merge_replicates.py \
                   -i {':'.join(input_files)} -o {output_file} \
                   -m {method} -c {condition_name}"""
    
    result = run_command(merge_cmd, f"Merging {file_pattern} files for {condition_name}", check=False)
    
    if result and result.returncode == 0:
        logger.info(f"✅ {condition_name}: {file_pattern}ファイルマージ完了 (N={len(input_files)})")
        return True
    else:
        logger.error(f"❌ {condition_name}: {file_pattern}ファイルマージ失敗")
        return False

def detect_outliers_in_condition(condition_name, file_pattern='rpkm'):
    """
    同一条件内でのreplicate外れ値検出
    
    Args:
        condition_name (str): 実験条件名
        file_pattern (str): 解析対象ファイルパターン
    
    Returns:
        list: 外れ値と判定されたreplicate ID一覧
    """
    if not EXPERIMENT_DESIGN['replicate_handling']['outlier_detection']:
        return []
    
    replicates = get_condition_replicates(condition_name)
    if len(replicates) < 3:  # 外れ値検出には最低3サンプル必要
        return []
    
    # 簡易統計解析による外れ値検出のロジックを実装
    # （実際の実装では、より詳細な統計解析が必要）
    outliers = []
    
    logger.info(f"📊 {condition_name}: 外れ値検出実行中 (N={len(replicates)})")
    # ここに統計解析ロジックを追加
    
    return outliers

def validate_replicate_structure():
    """
    実験設計のreplicate構造を検証
    
    Returns:
        dict: 検証結果
    """
    validation_results = {
        'valid': True,
        'warnings': [],
        'errors': [],
        'summary': {}
    }
    
    for condition_name, condition_info in EXPERIMENT_DESIGN['conditions'].items():
        replicate_count = len(condition_info['replicates'])
        validation_results['summary'][condition_name] = {
            'replicate_count': replicate_count,
            'type': condition_info['type']
        }
        
        if replicate_count < EXPERIMENT_DESIGN['replicate_handling']['min_replicates']:
            validation_results['warnings'].append(
                f"{condition_name}: replicate数が最小要件を下回っています (N={replicate_count})"
            )
        
        # ファイル存在確認
        missing_files = []
        for replicate_id in condition_info['replicates']:
            if replicate_id in INPUT_FASTQ_FILES:
                r1_path = INPUT_FASTQ_FILES[replicate_id]['R1']
                r2_path = INPUT_FASTQ_FILES[replicate_id]['R2']
                if not Path(r1_path).exists():
                    missing_files.append(f"{replicate_id}_R1")
                if not Path(r2_path).exists():
                    missing_files.append(f"{replicate_id}_R2")
        
        if missing_files:
            validation_results['errors'].append(
                f"{condition_name}: FASTQファイルが見つかりません: {', '.join(missing_files)}"
            )
            validation_results['valid'] = False
    
    return validation_results

def find_latest_processed_files(sample_name, file_type="fastq"):
    """
    サンプルの最新処理済みファイルを動的に検索
    
    Args:
        sample_name (str): サンプル名
        file_type (str): ファイルタイプ ("fastq", "sam", "rt" など)
    
    Returns:
        dict: {"R1": path, "R2": path} または Path または None
    """
    search_patterns = {
        "fastq": [
            f"data/processed/trimmed/{sample_name}_trimmed",  # トリミング済み
            f"data/processed/{sample_name}_rmdup",            # PCR重複除去済み
            f"data/processed/{sample_name}_demux",            # デマルチプレックス済み
        ],
        "sam": [
            f"data/processed/aligned/{sample_name}.sam"
        ],
        "rt": [
            f"data/processed/aligned/{sample_name}.rt"
        ]
    }
    
    if file_type == "fastq":
        # ペアエンドファイルの場合
        for pattern in search_patterns[file_type]:
            r1_file = PROJECT_ROOT / f"{pattern}_1.fastq"
            r2_file = PROJECT_ROOT / f"{pattern}_2.fastq"
            
            if r1_file.exists() and r2_file.exists():
                logger.info(f"✅ {sample_name}: 使用ファイル {pattern}_*.fastq")
                return {"R1": r1_file, "R2": r2_file}
        
        # 最後の手段：元のFASTQファイル
        if sample_name in INPUT_FASTQ_FILES:
            original_r1 = INPUT_FASTQ_FILES[sample_name]['R1']
            original_r2 = INPUT_FASTQ_FILES[sample_name]['R2']
            if Path(original_r1).exists() and Path(original_r2).exists():
                logger.warning(f"⚠️ {sample_name}: 元ファイルを使用 {original_r1}, {original_r2}")
                return {"R1": Path(original_r1), "R2": Path(original_r2)}
            
        return None
    
    else:
        # 単一ファイルの場合
        for pattern in search_patterns.get(file_type, []):
            file_path = PROJECT_ROOT / pattern
            if file_path.exists():
                logger.info(f"✅ {sample_name}: 使用ファイル {pattern}")
                return file_path
        return None

def create_output_dirs():
    """出力ディレクトリ作成"""
    dirs = [
        'logs',  # ログディレクトリを追加
        'data/processed/fastqc',
        'data/processed/trimmed', 
        'data/processed/aligned',
        'data/processed/rbrp_scores',
        'data/processed/merged_conditions',  # replicate統合結果
        'data/results/bigwig',
        'data/results/figures',
        'data/results/statistical_analysis'  # 統計解析結果
    ]
    
    for dir_path in dirs:
        (PROJECT_ROOT / dir_path).mkdir(parents=True, exist_ok=True)
    
    logger.info("📁 出力ディレクトリを作成しました")

# 実験設計の検証実行
validation_results = validate_replicate_structure()
create_output_dirs()

print("📁 出力ディレクトリ準備完了")

# クリーンアップ設定の表示
if PROCESSING_OPTIONS.get('cleanup_intermediate_fastq', False):
    print("💾 ディスク容量管理: ✅ 中間FASTQファイル自動削除有効")
    print(f"🔒 元ファイル保護: {'✅ 有効' if PROCESSING_OPTIONS.get('preserve_original_fastq', True) else '❌ 無効'}")
else:
    print("💾 ディスク容量管理: ❌ 中間FASTQファイル削除無効（手動削除が必要）")

print("\n🔬 実験設計検証結果:")
if validation_results['valid']:
    print("✅ 実験設計は有効です")
else:
    print("❌ 実験設計にエラーがあります")

for warning in validation_results['warnings']:
    print(f"⚠️ 警告: {warning}")

for error in validation_results['errors']:
    print(f"❌ エラー: {error}")

print(f"\n📊 実験設計サマリー:")
for condition, info in validation_results['summary'].items():
    print(f"  🧪 {condition}: N={info['replicate_count']} ({info['type']})")

## 5. ステップ1: 品質管理・デマルチプレックス

In [ ]:
def step1_quality_control_and_demultiplex():
    """ステップ1: 品質管理とデマルチプレックス（スキップオプション対応）"""
    print("\n🔍 ステップ1: 品質管理・デマルチプレックス開始")
    
    # スキップオプションの確認
    skip_demux = PROCESSING_OPTIONS.get('skip_demultiplex', False)
    perform_qc = PROCESSING_OPTIONS.get('perform_quality_control', True)
    
    if skip_demux:
        print("🚀 デマルチプレックスをスキップ（最近のシーケンサーでは既に実行済み）")
    if not perform_qc:
        print("🚀 品質管理をスキップ")
    
    for sample_name, fastq_paths in tqdm(INPUT_FASTQ_FILES.items(), desc="Processing samples"):
        # ペアエンドファイルの存在確認
        r1_path = fastq_paths['R1']
        r2_path = fastq_paths['R2']
        
        if not Path(r1_path).exists():
            logger.warning(f"⚠️ R1ファイルが見つかりません: {r1_path}")
            continue
        if not Path(r2_path).exists():
            logger.warning(f"⚠️ R2ファイルが見つかりません: {r2_path}")
            continue
        
        # FastQC実行（オプション）
        if perform_qc:
            fastqc_cmd_r1 = f"fastqc -o {PROJECT_ROOT}/data/processed/fastqc {r1_path}"
            fastqc_cmd_r2 = f"fastqc -o {PROJECT_ROOT}/data/processed/fastqc {r2_path}"
            
            run_command(fastqc_cmd_r1, f"FastQC for {sample_name} R1")
            run_command(fastqc_cmd_r2, f"FastQC for {sample_name} R2")
        else:
            print(f"⏭️ {sample_name}: FastQCをスキップしました")
        
        # デマルチプレックス（オプション）
        if not skip_demux and sample_name in BARCODES:
            barcode = BARCODES[sample_name]
            output_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
            output_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
            
            # ペアエンドバーコード抽出
            demux_cmd_r1 = f"""grep -A 3 "^@.*{barcode}" {r1_path} | grep -v "^--$" > {output_r1}"""
            demux_cmd_r2 = f"""grep -A 3 "^@.*{barcode}" {r2_path} | grep -v "^--$" > {output_r2}"""
            
            run_command(demux_cmd_r1, f"Demultiplexing {sample_name} R1", check=False)
            run_command(demux_cmd_r2, f"Demultiplexing {sample_name} R2", check=False)
            
            logger.info(f"✅ {sample_name}: ペアエンドデマルチプレックス完了")
        elif skip_demux:
            # デマルチプレックスをスキップする場合、元ファイルへのシンボリックリンクを作成
            output_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
            output_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
            
            # シンボリックリンク作成（既に存在する場合は削除）
            if output_r1.exists():
                output_r1.unlink()
            if output_r2.exists():
                output_r2.unlink()
                
            output_r1.symlink_to(Path(r1_path).absolute())
            output_r2.symlink_to(Path(r2_path).absolute())
            
            logger.info(f"✅ {sample_name}: デマルチプレックスをスキップ（シンボリックリンク作成）")
        else:
            logger.warning(f"⚠️ {sample_name}: バーコードが指定されていません")
    
    skip_msg = []
    if skip_demux:
        skip_msg.append("デマルチプレックス")
    if not perform_qc:
        skip_msg.append("品質管理")
    
    completion_msg = "✅ ステップ1完了: 品質管理・デマルチプレックス"
    if skip_msg:
        completion_msg += f"（{', '.join(skip_msg)}をスキップ）"
    
    print(completion_msg)

step1_quality_control_and_demultiplex()

## 6. ステップ2: PCR重複除去・トリミング

In [ ]:
def step2_remove_duplicates_and_trim():
    """ステップ2: PCR重複除去とトリミング（動的ファイル解決対応・ディスク容量管理）"""
    print("\n✂️ ステップ2: PCR重複除去・トリミング開始（動的ファイル解決対応・ディスク容量管理）")
    
    # スキップオプションの確認
    skip_trimming = PROCESSING_OPTIONS.get('skip_adapter_trimming', False)
    skip_pcr_removal = PROCESSING_OPTIONS.get('skip_pcr_duplicate_removal', False)
    
    if skip_trimming:
        print("🚀 アダプタートリミングをスキップ（最近のシーケンサーでは既に実行済み）")
    if skip_pcr_removal:
        print("🚀 PCR重複除去をスキップ")
    
    # 全体のクリーンアップ統計
    total_cleanup_stats = {
        'files_removed': 0,
        'space_freed_mb': 0,
        'preserved_originals': 0
    }
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing trimming"):
        # 前ステップのファイルを動的に検索（デマルチプレックス済み）
        input_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
        input_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
        
        if not input_r1.exists() or not input_r2.exists():
            logger.warning(f"⚠️ {sample_name}: デマルチプレックス済みファイルが見つかりません")
            # 元ファイルの確認
            original_r1 = Path(INPUT_FASTQ_FILES[sample_name]['R1'])
            original_r2 = Path(INPUT_FASTQ_FILES[sample_name]['R2'])
            if original_r1.exists() and original_r2.exists():
                input_r1, input_r2 = original_r1, original_r2
                logger.info(f"📄 {sample_name}: 元ファイルを使用")
            else:
                logger.error(f"❌ {sample_name}: 入力ファイルが見つかりません")
                continue
        
        # PCR重複除去（オプション）
        files_to_cleanup_after_pcr = []
        if not skip_pcr_removal:
            rmdup_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_rmdup_1.fastq"
            rmdup_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_rmdup_2.fastq"
            
            # ペアエンドPCR重複除去（簡易版）
            rmdup_cmd_r1 = f"""awk '/^@/ {{if (seen[$0]++) next}} 1' {input_r1} > {rmdup_r1}"""
            rmdup_cmd_r2 = f"""awk '/^@/ {{if (seen[$0]++) next}} 1' {input_r2} > {rmdup_r2}"""
            
            run_command(rmdup_cmd_r1, f"Remove duplicates for {sample_name} R1")
            run_command(rmdup_cmd_r2, f"Remove duplicates for {sample_name} R2")
            
            # 次のステップの入力ファイルを更新
            trim_input_r1 = rmdup_r1
            trim_input_r2 = rmdup_r2
            
            # デマルチプレックス済みファイルをクリーンアップ対象に追加（元ファイルでない場合）
            if not is_original_fastq_file(input_r1, sample_name):
                files_to_cleanup_after_pcr.append(input_r1)
            if not is_original_fastq_file(input_r2, sample_name):
                files_to_cleanup_after_pcr.append(input_r2)
        else:
            print(f"⏭️ {sample_name}: PCR重複除去をスキップしました")
            trim_input_r1 = input_r1
            trim_input_r2 = input_r2
        
        # アダプタートリミング（オプション）
        trimmed_r1 = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed_1.fastq"
        trimmed_r2 = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed_2.fastq"
        
        files_to_cleanup_after_trim = []
        if not skip_trimming:
            # ペアエンドアダプター除去とクオリティトリミング
            adapter_r1 = PROCESSING_OPTIONS['adapter_sequences']['R1']
            adapter_r2 = PROCESSING_OPTIONS['adapter_sequences']['R2']
            
            trim_cmd = f"""cutadapt -a {adapter_r1} -A {adapter_r2} \
                          -q 30 -m {ANALYSIS_PARAMS['min_read_length']} \
                          -o {trimmed_r1} -p {trimmed_r2} \
                          {trim_input_r1} {trim_input_r2}"""
            
            run_command(trim_cmd, f"Paired-end adapter trimming for {sample_name}", check=False)
            
            # PCR重複除去済みファイルをクリーンアップ対象に追加
            if not skip_pcr_removal:
                files_to_cleanup_after_trim.extend([trim_input_r1, trim_input_r2])
        else:
            # トリミングをスキップする場合、シンボリックリンクを作成
            if trimmed_r1.exists():
                trimmed_r1.unlink()
            if trimmed_r2.exists():
                trimmed_r2.unlink()
                
            trimmed_r1.symlink_to(trim_input_r1.absolute())
            trimmed_r2.symlink_to(trim_input_r2.absolute())
            
            print(f"⏭️ {sample_name}: アダプタートリミングをスキップ（シンボリックリンク作成）")
        
        # PCR重複除去後のクリーンアップ
        if files_to_cleanup_after_pcr:
            cleanup_stats = safe_cleanup_fastq_files(
                files_to_cleanup_after_pcr, 
                sample_name, 
                f"PCR重複除去完了後 ({sample_name})"
            )
            if not cleanup_stats.get('skipped', False):
                total_cleanup_stats['files_removed'] += cleanup_stats['files_removed']
                total_cleanup_stats['space_freed_mb'] += cleanup_stats['space_freed_mb']
                total_cleanup_stats['preserved_originals'] += cleanup_stats['preserved_originals']
        
        # トリミング後のクリーンアップ
        if files_to_cleanup_after_trim:
            cleanup_stats = safe_cleanup_fastq_files(
                files_to_cleanup_after_trim, 
                sample_name, 
                f"トリミング完了後 ({sample_name})"
            )
            if not cleanup_stats.get('skipped', False):
                total_cleanup_stats['files_removed'] += cleanup_stats['files_removed']
                total_cleanup_stats['space_freed_mb'] += cleanup_stats['space_freed_mb']
                total_cleanup_stats['preserved_originals'] += cleanup_stats['preserved_originals']
        
        logger.info(f"✅ {sample_name}: 処理完了")
    
    # 完了メッセージ作成
    skip_msg = []
    if skip_trimming:
        skip_msg.append("アダプタートリミング")
    if skip_pcr_removal:
        skip_msg.append("PCR重複除去")
    
    completion_msg = "✅ ステップ2完了: PCR重複除去・トリミング（動的ファイル解決対応・ディスク容量管理）"
    if skip_msg:
        completion_msg += f"（{', '.join(skip_msg)}をスキップ）"
    
    print(completion_msg)
    
    # クリーンアップサマリー表示
    if PROCESSING_OPTIONS.get('cleanup_intermediate_fastq', False) and total_cleanup_stats['files_removed'] > 0:
        print(f"💾 ステップ2クリーンアップサマリー:")
        print(f"   🗑️ 削除ファイル数: {total_cleanup_stats['files_removed']}")
        print(f"   💾 節約容量: {total_cleanup_stats['space_freed_mb']:.1f} MB")
        print(f"   🔒 保護した元ファイル: {total_cleanup_stats['preserved_originals']}")

step2_remove_duplicates_and_trim()

## 7. ステップ3: 配列アライメント・転写産物発現量計算

In [ ]:
def step3_alignment_and_expression():
    """ステップ3: 配列アライメントと転写産物発現量計算（動的ファイル解決対応）"""
    print("\n🧬 ステップ3: アライメント・発現量計算開始（動的ファイル解決対応）")
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing alignment"):
        # 動的に最新の処理済みFASTQファイルを検索
        fastq_files = find_latest_processed_files(sample_name, "fastq")
        
        if not fastq_files:
            logger.error(f"❌ {sample_name}: 処理済みFASTQファイルが見つかりません")
            continue
            
        trimmed_r1 = fastq_files["R1"]
        trimmed_r2 = fastq_files["R2"]
        
        if not trimmed_r1.exists():
            logger.warning(f"⚠️ R1ファイルが見つかりません: {trimmed_r1}")
            continue
        if not trimmed_r2.exists():
            logger.warning(f"⚠️ R2ファイルが見つかりません: {trimmed_r2}")
            continue
        
        # Bowtie2でペアエンドアライメント
        sam_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.sam"
        
        # ペアエンドマッピング（-1, -2でペアエンドファイル指定）
        align_cmd = f"""bowtie2 -1 {trimmed_r1} -2 {trimmed_r2} -S {sam_file} \
                       -x {REFERENCE_FILES['transcriptome_index']} \
                       --non-deterministic --time \
                       --minins 50 --maxins 500 \
                       --no-mixed --no-discordant"""
        
        run_command(align_cmd, f"Paired-end alignment for {sample_name}")
        
        # RPKM計算
        rpkm_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rpkm"
        rpkm_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rpkm.py -i {sam_file} -o {rpkm_file}"""
        
        run_command(rpkm_cmd, f"RPKM calculation for {sample_name}", check=False)
        
        # RTstop計算
        rt_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rt"
        rt_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rtstops.py \
                    -i {sam_file} -o {rt_file} -r {rpkm_file} -c {ANALYSIS_PARAMS['min_rpkm']}"""
        
        run_command(rt_cmd, f"RTstop calculation for {sample_name}", check=False)
        logger.info(f"✅ {sample_name}: ペアエンドアライメント・発現量計算完了")
    
    print("✅ ステップ3完了: アライメント・発現量計算（動的ファイル解決対応）")

step3_alignment_and_expression()

## 8. ステップ4: RBRPスコア計算・統計解析

In [ ]:
def step4_rbrp_score_calculation():
    """ステップ4: RBRPスコア計算と統計解析（Replicate対応・動的ファイル解決）"""
    print("\n📊 ステップ4: RBRPスコア計算・統計解析開始（Replicate対応・動的ファイル解決）")
    
    # 動的にサンプルグループを取得
    probe_samples = get_samples_by_type('probe')
    control_samples = get_samples_by_type('control')
    
    print(f"📋 プローブサンプル: {probe_samples}")
    print(f"📋 コントロールサンプル: {control_samples}")
    
    # まず個別replicateのRBRPスコア計算
    print("\n🔬 個別replicateのRBRPスコア計算:")
    
    # バックグラウンドRTstopファイルをマージ（コントロールサンプル）
    if len(control_samples) >= 2:
        control_files = []
        for sample in control_samples:
            rt_file = find_latest_processed_files(sample, "rt")
            if rt_file and rt_file.exists():
                control_files.append(str(rt_file))
            else:
                logger.warning(f"⚠️ {sample}: RTファイルが見つかりません")
        
        if control_files:
            merged_control = PROJECT_ROOT / "data/processed/rbrp_scores/merged_control.rt"
            merge_cmd = f"""python {PROJECT_ROOT}/scripts/merge_rt_files.py \
                           -i {':'.join(control_files)} -o {merged_control}"""
            run_command(merge_cmd, "Merging control RTstop files", check=False)
    
    # 各プローブサンプルのRBRPスコア計算
    for sample_name in tqdm(probe_samples, desc="Calculating individual RBRP scores"):
        rt_file = find_latest_processed_files(sample_name, "rt")
        
        if not rt_file or not rt_file.exists():
            logger.warning(f"⚠️ {sample_name}: RTファイルが見つかりません")
            continue
        
        # RTファイル正規化
        normalized_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_normalized.rt"
        norm_cmd = f"""python {PROJECT_ROOT}/scripts/normalize_rt_file.py \
                      -i {rt_file} -o {normalized_file} -d 32 -l 32"""
        run_command(norm_cmd, f"Normalizing RT file for {sample_name}", check=False)
        
        # RBRPスコア計算
        rbrp_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_rbrp.out"
        background_file = merged_control if 'merged_control' in locals() else None
        
        if background_file and background_file.exists():
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -b {background_file} -o {rbrp_file} \
                          -e dividing -y 0.5"""
        else:
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -o {rbrp_file}"""
        
        run_command(rbrp_cmd, f"Calculating RBRP scores for {sample_name}", check=False)
        
        # 低品質スコアフィルタリング
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        filter_cmd = f"""python {PROJECT_ROOT}/scripts/filter_rbrp_scores.py \
                        -i {rbrp_file} -o {filtered_file} \
                        -t {ANALYSIS_PARAMS['min_sequencing_depth']} -s 5 -e 30"""
        run_command(filter_cmd, f"Filtering RBRP scores for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: RBRPスコア計算完了")
    
    # Replicate統合処理
    if EXPERIMENT_DESIGN['replicate_handling']['merge_replicates']:
        print("\n🔗 Replicateマージ処理:")
        
        for condition_name, condition_info in EXPERIMENT_DESIGN['conditions'].items():
            if condition_info['type'] != 'probe':
                continue  # プローブサンプルのみ処理
                
            replicate_count = len(condition_info['replicates'])
            print(f"\n📊 {condition_name} (N={replicate_count}):")
            
            # 外れ値検出
            if EXPERIMENT_DESIGN['replicate_handling']['outlier_detection']:
                outliers = detect_outliers_in_condition(condition_name, 'rbrp')
                if outliers:
                    print(f"⚠️ 外れ値検出: {outliers}")
            
            # RTファイルマージ
            merged_rt_file = PROJECT_ROOT / f"data/processed/merged_conditions/{condition_name}_merged.rt"
            if merge_replicate_files(condition_name, 'rt', merged_rt_file, 
                                   EXPERIMENT_DESIGN['replicate_handling']['statistical_method']):
                print(f"✅ RTファイルマージ完了: {condition_name}")
            
            # RPKMファイルマージ
            merged_rpkm_file = PROJECT_ROOT / f"data/processed/merged_conditions/{condition_name}_merged.rpkm"
            if merge_replicate_files(condition_name, 'rpkm', merged_rpkm_file,
                                   EXPERIMENT_DESIGN['replicate_handling']['statistical_method']):
                print(f"✅ RPKMファイルマージ完了: {condition_name}")
            
            # RBRPスコアマージ
            merged_rbrp_file = PROJECT_ROOT / f"data/processed/merged_conditions/{condition_name}_merged_rbrp.out"
            if merge_replicate_files(condition_name, 'rbrp', merged_rbrp_file,
                                   EXPERIMENT_DESIGN['replicate_handling']['statistical_method']):
                print(f"✅ RBRPスコアマージ完了: {condition_name}")
                
                # マージ済みファイルの統計サマリー生成
                stats_file = PROJECT_ROOT / f"data/results/statistical_analysis/{condition_name}_replicate_stats.txt"
                stats_cmd = f"""python {PROJECT_ROOT}/scripts/generate_replicate_stats.py \
                               -c {condition_name} -n {replicate_count} \
                               -m {EXPERIMENT_DESIGN['replicate_handling']['statistical_method']} \
                               -o {stats_file}"""
                run_command(stats_cmd, f"Generating replicate statistics for {condition_name}", check=False)
    
    # 個別replicate結果の保持
    if EXPERIMENT_DESIGN['replicate_handling']['keep_individual_results']:
        print("\n📦 個別replicate結果保持: ✅ 有効")
    else:
        print("\n📦 個別replicate結果保持: ❌ 無効（クリーンアップ実行）")
        # ここでクリーンアップロジックを追加可能
    
    print("✅ ステップ4完了: RBRPスコア計算・統計解析（Replicate対応・動的ファイル解決）")

step4_rbrp_score_calculation()

## 9. ステップ5: 可視化ファイル生成・結果出力

In [ ]:
def step5_visualization_and_output():
    """ステップ5: 可視化ファイル生成と結果出力"""
    print("\n📈 ステップ5: 可視化ファイル生成・結果出力開始")
    
    # プローブサンプルを定義（SRA accession numbers基準）
    probe_samples = [
        'SRR22397001',  # HEK293_Probe2_only_rep1
        'SRR22397002',  # HEK293_Probe2_only_rep2
        'SRR22397003',  # HEK293_Probe2_Levofloxacin_rep1
        'SRR22397004'   # HEK293_Probe2_Levofloxacin_rep2
    ]
    
    for sample_name in tqdm(probe_samples, desc="Generating visualization files"):
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        
        if not filtered_file.exists():
            logger.warning(f"⚠️ フィルタリング済みファイルが見つかりません: {filtered_file}")
            continue
        
        # bedgraphファイル生成
        bedgraph_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bedgraph"
        bedgraph_cmd = f"""python {PROJECT_ROOT}/scripts/generate_bedgraph.py \
                          -i {filtered_file} -o {bedgraph_file} \
                          -g {REFERENCE_FILES['genome_gtf']} \
                          -a {REFERENCE_FILES.get('transcriptome_fasta', '')}"""
        run_command(bedgraph_cmd, f"Generating bedgraph for {sample_name}", check=False)
        
        # bigwigファイル生成（UCscツールが利用可能な場合）
        bigwig_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bw"
        genome_size_file = PROJECT_ROOT / "data/genome.size"  # 事前に準備が必要
        
        if bedgraph_file.exists():
            # ソートと重複除去
            sorted_bedgraph = PROJECT_ROOT / f"data/results/bigwig/{sample_name}_sorted.bedgraph"
            sort_cmd = f"sort -k1,1 -k2,3n {bedgraph_file} | uniq > {sorted_bedgraph}"
            run_command(sort_cmd, f"Sorting bedgraph for {sample_name}", check=False)
            
            # bigwig変換
            if genome_size_file.exists():
                bw_cmd = f"bedGraphToBigWig {sorted_bedgraph} {genome_size_file} {bigwig_file}"
                run_command(bw_cmd, f"Converting to bigwig for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: 可視化ファイル生成完了")
    
    print("✅ ステップ5完了: 可視化ファイル生成・結果出力")

step5_visualization_and_output()

## 10. 結果サマリー・統計情報

In [ ]:
# 実行時間の記録
print(f"\n🎉 Replicate対応動的ファイル解決パイプライン実行完了!")
print(f"📁 結果ファイルは以下に保存されました:")
print(f"   - 個別処理済みデータ: {PROJECT_ROOT}/data/processed/")
print(f"   - 条件別マージデータ: {PROJECT_ROOT}/data/processed/merged_conditions/")
print(f"   - 統計解析結果: {PROJECT_ROOT}/data/results/statistical_analysis/")
print(f"   - 最終結果: {PROJECT_ROOT}/data/results/")
print(f"   - ログファイル: {log_file}")

# ディスク容量管理サマリー
if PROCESSING_OPTIONS.get('cleanup_intermediate_fastq', False):
    print(f"\n💾 ディスク容量管理サマリー:")
    print(f"   🗑️ 中間FASTQファイル自動削除: ✅ 有効")
    print(f"   🔒 元ファイル保護: ✅ 有効")
    print(f"   📊 推定容量節約: 大きなFASTQファイルの場合、数GB〜数十GB節約可能")
    print(f"   ⚠️ 注意: 元のダウンロード/シーケンスファイルは保護されます")
else:
    print(f"\n💾 ディスク容量管理:")
    print(f"   🗑️ 中間FASTQファイル削除: ❌ 無効")
    print(f"   ⚠️ 手動クリーンアップが必要な場合があります")
    
    # 手動クリーンアップの提案
    print(f"\n📋 手動クリーンアップが必要な中間ファイル:")
    for sample_name in INPUT_FASTQ_FILES.keys():
        potential_cleanup_files = [
            f"data/processed/{sample_name}_demux_*.fastq",
            f"data/processed/{sample_name}_rmdup_*.fastq",
            f"data/processed/trimmed/{sample_name}_trimmed_*.fastq"
        ]
        for pattern in potential_cleanup_files:
            print(f"   - {pattern}")

logger.info("RBRP Dry Protocol Pipeline (Replicate-aware Dynamic File Resolution with Disk Management) completed successfully")

## 11. 結果可視化（オプション）

In [ ]:
def create_visualization_plots():
    """結果の可視化プロット作成"""
    print("\n📊 結果可視化プロット作成")
    
    # 処理サマリーの可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('RBRP Dry Protocol - 処理結果サマリー', fontsize=16, y=0.98)
    
    # 1. ファイルサイズ分布
    size_columns = [col for col in summary_report.columns if 'size_mb' in col]
    if size_columns:
        size_data = summary_report[size_columns].fillna(0)
        axes[0, 0].bar(range(len(size_columns)), size_data.mean(), 
                      tick_label=[col.replace('_size_mb', '') for col in size_columns])
        axes[0, 0].set_title('平均ファイルサイズ (MB)')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. 処理成功率
    success_columns = [col for col in summary_report.columns if 'exists' in col]
    if success_columns:
        success_rates = summary_report[success_columns].mean() * 100
        axes[0, 1].bar(range(len(success_rates)), success_rates.values,
                      tick_label=[col.replace('_exists', '') for col in success_columns])
        axes[0, 1].set_title('処理成功率 (%)')
        axes[0, 1].set_ylim(0, 100)
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. サンプル別処理状況
    if success_columns:
        sample_success = summary_report[success_columns].sum(axis=1)
        axes[1, 0].bar(range(len(sample_success)), sample_success.values,
                      tick_label=summary_report['sample_name'])
        axes[1, 0].set_title('サンプル別完了ステップ数')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. 処理時間の目安（仮想データ）
    processing_steps = ['デマルチプレックス', 'トリミング', 'アライメント', 'RBRP計算', '可視化']
    estimated_times = [5, 10, 30, 20, 5]  # 分単位
    axes[1, 1].bar(processing_steps, estimated_times)
    axes[1, 1].set_title('ステップ別推定処理時間 (分)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    # 図を保存
    plot_file = PROJECT_ROOT / "data/results/figures/processing_summary.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 可視化プロット保存: {plot_file}")

# 可視化実行
try:
    create_visualization_plots()
except Exception as e:
    logger.warning(f"可視化プロット作成エラー: {e}")
    print("⚠️ 可視化プロットの作成でエラーが発生しましたが、パイプライン自体は正常に完了しています")